In [1]:
import csv

# Functions

## read_input_poind_config

#### Outline

- `\Zone.csv`
  - $1$行目はヘッダ：地域区分, 都市, 緯度, 経度, 日射量ファイル名, 暖房開始日, 暖房終了日, 冷房開始日, 冷房終了日

  - $2～9$行目：$1～8$地域の「地域区分, 都市, 緯度, 経度, 日射量ファイル名, 暖房開始日, 暖房終了日, 冷房開始日, 冷房終了日」
    - 暖冷房開始日終了日の書式：$5$桁もしくは$6$桁の数字
      - 気象データの日時表記と同じ
      - 後ろから$2$桁：時刻
      - 後ろから$3～4$桁：日
      - 後ろから$5～6$桁：月
      - 気象データの「暖房$1$冷房$2$」の設定は上書き処理される → `\Zone.csv` の設定が優先

地点データを \Zone.csv から読み込む  
\地域区分+日射量データ窓面入射角特性.xlsx "地域区分"シート
→ \SCFConfig01 下の地点データファイル \Zone.csv を作成 → 読み込み

#### Function

In [2]:
def input_Point(tag):
    
    def convert_tag_to_city(tag):
        if type(tag) == int: # name に数字が指定された場合は省エネ基準の地域区分番号とみなして該当する都市に置き換える
            return {
                '1' : '北見',
                '2' : '岩見沢',
                '3' : '盛岡',
                '4' : '長野',
                '5' : '宇都宮',
                '6' : '岡山',
                '7' : '宮崎',
                '8' : '那覇'
            }[str(tag)]
        else: # その他の場合は都市名が入力されているものとし、変換は行わない
            return tag
    
    ZONE_FILE = "./SCFConfig01/Zone.csv"
    
    city_name = convert_tag_to_city(tag)

    with open(ZONE_FILE, 'r') as f:
        
        reader = csv.reader(f)
        header = next(reader) # Skip header.

        matched_rows = [ row for row in reader if row[1] == city_name ]  # Extract row matched.
        
        if len(matched_rows) == 0:  # In the case that there is now matched data.
            raise IndexError("地点データがありません")
        elif len(matched_rows) > 1:  # In the case that there are more than 1 matched data.
            raise IndexError("該当する地点データが複数見つかりました")
        else:
            matched_row = matched_rows[0]
            return [
                int(matched_row[0]), # region
                matched_row[1],      # city name
                float(matched_row[2]), # latitude
                float(matched_row[3]), # longitude
                matched_row[4],      # file name
                int(matched_row[5]), # start of heating period
                int(matched_row[6]), # end of heating period
                int(matched_row[7]), # start of cooling period
                int(matched_row[8])  # end of cooling period
            ]
            # ここで返るSRFileNameはファイル名のみ

#### Example 1

省エネルギー基準における地域区分の番号で指定した場合

In [3]:
for i in range(1,9):
    print( input_Point( i ) )

[1, '北見', 43.82, 143.91, 'SRforSCF_01.csv', 110100, 43024, 50100, 103124]
[2, '岩見沢', 43.21, 141.78833, 'SRforSCF_02.csv', 110100, 43024, 50100, 103124]
[3, '盛岡', 39.695, 141.168333333333, 'SRforSCF_03.csv', 120100, 33124, 40100, 113024]
[4, '長野', 36.66, 138.195, 'SRforSCF_04.csv', 120100, 33124, 40100, 113024]
[5, '宇都宮', 36.5466666666667, 139.871666666667, 'SRforSCF_05.csv', 120100, 33124, 40100, 113024]
[6, '岡山', 34.6583333333333, 133.918333333333, 'SRforSCF_06.csv', 120100, 33124, 40100, 113024]
[7, '宮崎', 31.935, 131.416666666667, 'SRforSCF_07.csv', 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 'SRforSCF_08.csv', 10100, 33124, 40100, 123124]


#### Example 2

都市名で指定した場合

In [4]:
print(input_Point( "宇都宮"))
print(input_Point( "那覇" ))

[5, '宇都宮', 36.5466666666667, 139.871666666667, 'SRforSCF_05.csv', 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 'SRforSCF_08.csv', 10100, 33124, 40100, 123124]


#### Example 3

都市名で指定したが該当する名前がリストになかった場合

In [5]:
try:
    input_Point( "東京" ) #データなし
except IndexError as e:
    print(e)

地点データがありません
